# Step 0 - Install Docker for Windows
### Download the installer for Docker from Windows from this [link](https://hub.docker.com/editions/community/docker-ce-desktop-windows/).

### Once the download completes, follow the instructions [here](https://docs.docker.com/docker-for-windows/install/) to install Docker.  Once the install completes, you will be ready to continue.

# Step 1 - To start the process, pull the latest image of SQL Server 2019 for Ubuntu.
### You can get full list of SQL Server Docker images from [Docker Hub](https://hub.docker.com/_/microsoft-mssql-server).  Images are available for each release of SQL Server 2017 and 2019 from the GA version up to the most recent CU.



In [ ]:
docker pull mcr.microsoft.com/mssql/server:2019-latest

## Step 2 - Create a SQL Server container using the latest image.  
### The script below creates a Docker container using the latest version of SQL Server 2019 running on Ubuntu Linus.  The script uses the following variables:

* $password - The sa password for the container and instance.
* $containername - The name of your container.  If a name is not provided Docker will create one for you.
* $port - The local port you will use to connect to the SQL Server instance.  This local port will be mapped to port 1433 in the container.
* $image - The image you pulled in the previous step.

### The docker run command contains parameters to accept to end user license agreement for SQL Server and to enable SQL Server Agent.




In [ ]:
$password = 'JuanS0t022';
$containername = 'sqlserver2019';
$port = 1436;
$image = 'mcr.microsoft.com/mssql/server:2019-latest';
docker run -e 'ACCEPT_EULA=Y' -e "SA_PASSWORD=$password" -e MSSQL_AGENT_ENABLED=True --name $containername -p "$port`:1433" -d $image;


# Step 3 - Copy WideWorldImporters backup to the container 
### Create a backup directory in your container.  Copy the backup file for WideWorldImporters to that directory. 

In [ ]:
docker exec sqlserver2019 mkdir /var/opt/mssql/backup
docker cp wwi.bak sqlserver2019:/var/opt/mssql/backup

# Step 4 - Check the version of SQL Server running on the container
### Use Invoke-Sqlcmd to run SELECT @@VERSION against the container.

In [ ]:
$cred = Get-Credential -UserName sa -Message "Enter the password for the sa account."
Invoke-Sqlcmd -ServerInstance "localhost,1436" -Database master -Credential $cred -Query "SELECT @@VERSION;" | Out-GridView;

# Step 5 - Restore the WideWorldImporters database copied in step 3
### Use Invoke-Sqlcmd to restore the WideWorldImporters database from the backup file copied in step 3.  The $cred variable from step 4 can be reused.

In [ ]:
<# Restore WideWorldImporters using Invoke-Sqlcmd #>
$query = @"
USE [master]
RESTORE DATABASE [WideWorldImporters] 
FROM  DISK = N'/var/opt/mssql/backup/wwi.bak' 
WITH  FILE = 1,  
MOVE N'WWI_Primary' TO N'/var/opt/mssql/data/WideWorldImporters.mdf',  
MOVE N'WWI_UserData' TO N'/var/opt/mssql/data/WideWorldImporters_UserData.ndf',  
MOVE N'WWI_Log' TO N'/var/opt/mssql/data/WideWorldImporters.ldf',  
MOVE N'WWI_InMemory_Data_1' TO N'/var/opt/mssql/data/WideWorldImporters_InMemory_Data_1',  
NOUNLOAD,  
REPLACE,
STATS = 5;
"@

Invoke-Sqlcmd -ServerInstance "localhost,1436" -Database master -Credential $cred -Query $query;

# Step 6 - Execute Ola Hallengren's Maintenance Solution
### Run the code block below to execute Ola Hallengren's Maintenance Solution on your container.  This will create backup jobs, index maintenance jobs, and integrity checking jobs.

In [5]:
$cred = Get-Credential -UserName sa -Message "Enter the password for the sa account"
Invoke-Sqlcmd -ServerInstance "localhost,1436" -Database master -Credential $cred -InputFile "C:\Users\fgill\Documents\GitHub\AzureDataStudioAndContainers\MaintenanceSolution.sql";

# Step 7 - Create schedules for the Maintenance Solution Jobs
### This will create the following schedules for the Maintenance Solution jobs.
* 

In [7]:
$cred = Get-Credential -UserName sa -Message "Enter the password for the sa account"
Invoke-Sqlcmd -ServerInstance "localhost,1436" -Database master -Credential $cred -InputFile "C:\Users\fgill\Documents\GitHub\AzureDataStudioAndContainers\ScheduleMaintenanceSolution.sql";


Column1
-------
     12
     13
     14
     15
     16


